<center><p float="center">
  <img src="https://mma.prnewswire.com/media/1458111/Great_Learning_Logo.jpg?p=facebook" width="200" height="100"/>
</p></center>

<h1><center><font size=10> Generative AI for NLP Program</center></font></h1>
<h1><center> Project </center></h1>

# **GA-NLP Mid-Term Project: Financial Product Complaint Classification and Summarization**

## **Business Context**

### **Description**
*In the modern financial industry, customer complaints play a crucial role in identifying areas where financial institutions can improve their services. Effectively categorizing these complaints into specific product categories, such as credit reports, student loans, or money transfers, is essential for addressing customer concerns promptly by routing the tickets to relevant personnel. Leveraging Generative AI for text classification can help financial institutions better understand customer grievances and respond more efficiently. Apart from this, a summary of the customer complaint helps the support personnel quickly grasp the gist of the grievance*

### **Objective**
*The primary goal of this project is to utilize Generative AI techniques to improve the classification and summarization of customer complaints in the financial sector.
Specifically, the project will focus on:*

1. **Text-to-Label Classification:** *Implementing Zero-shot and Few-shot prompting methods to accurately classify customer complaints into relevant product categories.*
2. **Text-to-Text Summarization:** *Using Zero-shot prompting to generate concise summaries of customer complaints, enabling more personalized and effective responses.*

### **Conclusion**
*Upon completing this project, you will have the capability to develop solutions for LLM-based text classification and summarization. These tools will enable financial institutions to automate the complaint handling process, leading to faster, more accurate responses to customer issues, improved customer satisfaction, and enhanced compliance with industry regulations. This project will also provide you with valuable skills and experience that can be applied to a range of real-world business challenges.*


# **Section 1 : Setting Up for Prompt Engineering with Mistral Model**

### **Install & Importing neccessary libraries**

In [1]:
!apt-get update
!apt-get install -y ninja-build cmake
!pip install ipywidgets --upgrade

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [2]:
import torch

In [3]:
# This part of code will skip all the un-necessary warnings which can occur during the execution of this project.
import warnings
warnings.filterwarnings("ignore", category=Warning)

In [4]:
# Installation for GPU llama-cpp-python==0.2.69
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69
# For downloading the models from HF Hub
!pip install huggingface_hub

  Using cached llama_cpp_python-0.2.69.tar.gz (42.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.69-cp311-cp311-linux_x86_64.whl size=55261174 sha256=232c0583132597c375441b79b56b88c3f67edb93abf437d467f0fede3b364035
  Stored in directory: /root/.cache/pip/wheels/e8/1b/ff/b4dba97fbd16e731705b262602ba8f3b672bf4bde54ea0c104
Successfully built llama-cpp-python


In [5]:
!pip install evaluate
!pip install bert-score

  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached dill-0.3.9-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.17-py311-none-any.whl.metadata (7.2 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached evaluate-0.4.3-py3-none-any.whl (84 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 

In [6]:
!pip freeze > requirement.txt

In [7]:
# Basic Imports for Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import re

import torch
import evaluate

# from google.colab import drive
import locale

### **Question 1: Importing Libaries and Mistral Model (3 Marks)**

- For the Mistral Model name or path and model basename, refer to the **Week 3 Additional Content: Prompt Engineering Fundamentals**
- Code Notebook: Self-Consistency and Tree-of-Thought Prompting with Llama 2 and Mistral.





https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/blob/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf

In [8]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [9]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q5_K_M.gguf"

In [10]:
model_path = hf_hub_download(
    repo_id="TheBloke/Mistral-7B-Instruct-v0.2-GGUF",
    filename="mistral-7b-instruct-v0.2.Q5_K_M.gguf"
    )

mistral-7b-instruct-v0.2.Q5_K_M.gguf:   0%|          | 0.00/5.13G [00:00<?, ?B/s]

In [11]:
lcpp_llm = Llama(
        model_path=model_path,
        n_threads=200,  # CPU cores
        n_batch=100,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
        n_gpu_layers=20,  # Change this value based on your model and your GPU VRAM pool.
        n_ctx=4096,  # Context window
    )

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loa

In [12]:
!nvidia-smi


Thu Jan 23 01:16:35 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              49W / 400W |   3803MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# **Section 2: Text to Label generation**

### **Question 2: Zero-Shot Prompting for Text Classification (5 Marks)**

##### **Q2.1: Define the Prompt Template, System Message, generate_prompt** **(2 Marks)**

- Define a **system message** as a string and assign it to the variable system_message to generate product class.
- Create a **zero shot prompt template** that incorporates the system message and user input.
- Define **generate_prompt** function that takes both the system_message and user_input as arguments and formats them into a prompt template


Write a Python function called **generate_mistral_response** that takes a single parameter, narrative, which represents the user's complain. Inside the function, you should perform the following tasks:


- **Combine the system_message and narrative to create a prompt string using generate_prompt function.**

*Generate a response from the Mistral model using the lcpp_llm instance with the following parameters:*

- prompt should be the combined prompt string.
- max_tokens should be set to 1200.
- temperature should be set to 0.
- top_p should be set to 0.95.
- repeat_penalty should be set to 1.2.
- top_k should be set to 50.
- stop should be set as a list containing '/s'.
- echo should be set to False.
Extract and return the response text from the generated response.

Don't forget to provide a value for the system_message variable before using it in the function.

In [13]:
system_message = "Classify the following customer complaint into categories such as 'Credit Card', 'Bank Account Services', 'Loan', 'Mortgage', and 'Others'. Also, provide a concise summary of the complaint."


In [14]:
zero_shot_prompt_template = """<s>[INST] Classify the following customer complaint into categories such as 'Credit Card', 'Bank Account Services', 'Loan', 'Mortgage', and 'Others'. Also, provide a concise summary of the complaint. [/INST]```{user_message}```"""

In [15]:
def generate_prompt(system_message,user_input):
    prompt=zero_shot_prompt_template.format(system_message=system_message,user_message=user_input)
    return prompt

In [16]:
def generate_mistral_response(input_text):
    # log the narrative being processed to monitor progress
    print(f'Processing narrative: {input_text[:100]}...')

    # Generate a response using the Mistral model
    response = lcpp_llm(
        prompt=generate_prompt(system_message, input_text),
        max_tokens=1200,
        temperature=0,
        top_p=0.95,
        top_k=50,
        stop=["/s"],
        repeat_penalty=1.2,
        echo=False
    )

    # Extract and return the response text
    return response["choices"][0]["text"]

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
# Load a CSV File containing Dataset of 500 products, narrative and summary (summary of narrative)
data = pd.read_csv('/content/drive/MyDrive/Generative AI/Complaints_classification.csv')

In [19]:
# Randomly select 30 rows
new_data = data.sample(n=30, random_state=40)

##### **Q2.2: Create a new column in the DataFrame called 'mistral_response' and populate it with responses generated by applying the 'generate_mistral_response' function to each 'narrative' in the DataFrame and prepare the mistral_response_cleaned column using extract_category function** **(1 Marks)**

In [20]:
def generate_mistral_response(input_text):
    print("\n---Processing new complaint---")
    print(f"Input: {input_text[:100]}...")

    prompt = generate_prompt(system_message, input_text)
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1200,
        temperature=0,
        top_p=0.95,
        top_k=50,
        stop=["/s"],
        repeat_penalty=1.2,
        echo=False
    )
    response_text = response["choices"][0]["text"]
    print(f"Response: {response_text}\n")
    return response_text

In [21]:
!nvidia-smi

Thu Jan 23 01:17:31 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              49W / 400W |   3803MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [22]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=4,        # Significantly reduced from 200
    n_batch=1,         # Reduced from 100
    n_gpu_layers=20,   # Keep this the same
    n_ctx=512,        # Significantly reduced from 4096
)

# Then try an even simpler test
test_response = lcpp_llm(
    prompt="Say hello",
    max_tokens=10,
    temperature=0,
    echo=False
)
print(test_response)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loa

{'id': 'cmpl-190dd5ab-ccfe-4ecb-8cda-5fe5d6551a29', 'object': 'text_completion', 'created': 1737595056, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q5_K_M.gguf', 'choices': [{'text': ' to the new and improved version of our popular ', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 4, 'completion_tokens': 10, 'total_tokens': 14}}


In [23]:
# Test with just one complaint
test_narrative = new_data['narrative'].iloc[0]
print("Testing classification for:")
print(test_narrative[:100], "...\n")

response = lcpp_llm(
    prompt=generate_prompt(system_message, test_narrative),
    max_tokens=200,  # Reduced from 1200 for testing
    temperature=0,
    top_p=0.95,
    stop=["/s"],
    echo=False
)
print("\nResponse:")
print(response["choices"][0]["text"])

Testing classification for:
fraudulent charge totaling made capital one checking account via debit card seeing charge immediatel ...



Llama.generate: prefix-match hit

llama_print_timings:        load time =   10772.41 ms
llama_print_timings:      sample time =      74.45 ms /   124 runs   (    0.60 ms per token,  1665.59 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   35325.09 ms /   390 runs   (   90.58 ms per token,    11.04 tokens per second)
llama_print_timings:       total time =   36333.12 ms /   391 tokens



Response:
 Based on the provided text, this customer complaint can be classified as 'Bank Account Services' with a sub-category of 'Fraud'. The customer is reporting an unauthorized use of their debit card and disputing a fraudulent charge made to their Capital One checking account. They have been in contact with Capital One multiple times to dispute the charge, but it has not yet been resolved. The customer believes that their debit card was intercepted and fraudulently activated, resulting in unauthorized purchases being made. They have also reported the incident to the local police department's financial services division.


In [24]:
def extract_category(text):
    # Define the regex pattern to match "category:" or "Category:" followed by a word
    pattern = r'category:\s*(\w+)'  # The pattern itself remains the same

    # Use re.search with the re.IGNORECASE flag to make it case-insensitive
    match = re.search(pattern, text, re.IGNORECASE)

    # If a match is found, return the captured group, else return None
    if match:
        return match.group(1)
    else:
        pattern1 = r'(credit_card|retail_banking|credit_reporting|mortgages_and_loans|debt_collection)'
        match = re.search(pattern1, text, re.IGNORECASE)
        if match:
            return match.group()
        else:
            return ''

def mistral_response_cleaned(text):
    # Example of cleaning or extracting information from the text
    cleaned_text = text.lower()  # Example transformation
    return cleaned_text

In [25]:
# Keep the current conservative parameters
# Reduce to 5 complaints
smaller_data = new_data.sample(n=5, random_state=42)

# Process one at a time with progress tracking
for idx, row in smaller_data.iterrows():
    print(f"\nProcessing complaint {idx+1}/5")
    try:
        response = generate_mistral_response(row['narrative'])
        smaller_data.loc[idx, 'mistral_response'] = response
        print(f"Successfully processed complaint {idx+1}")
    except Exception as e:
        print(f"Error processing complaint {idx+1}: {str(e)}")

# Only process cleaned responses for successful ones
smaller_data['mistral_response_cleaned'] = smaller_data['mistral_response'].apply(lambda x: mistral_response_cleaned(x))


Processing complaint 151/5

---Processing new complaint---
Input: true identity theft victim identity theft information listed appears credit report relate transactio...


Llama.generate: prefix-match hit

llama_print_timings:        load time =   10772.41 ms
llama_print_timings:      sample time =     140.30 ms /   232 runs   (    0.60 ms per token,  1653.60 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   35602.41 ms /   397 runs   (   89.68 ms per token,    11.15 tokens per second)
llama_print_timings:       total time =   36851.08 ms /   398 tokens
Llama.generate: prefix-match hit


Response:  Based on the given customer complaint, it appears that this is a case of Identity Theft. The customer mentions that they are a victim of identity theft and have found fraudulent transactions listed on their Equifax credit report. They also mention that they have been trying to get the issue resolved for some time but have encountered complications in the process.

Therefore, this complaint falls under the 'Others' category as it involves issues related to Identity Theft which is not directly linked to Credit Card, Bank Account Services, Loan or Mortgage services. However, since the identity theft has affected their credit report and ability to apply for new credit cards or loans in the future, it could potentially impact these areas as well.

The customer expresses frustration with the lengthy and complicated process of resolving the issue and the emotional and financial stress caused by the situation. They have also mentioned that they are seeking legal help and compensatio


llama_print_timings:        load time =   10772.41 ms
llama_print_timings:      sample time =      12.15 ms /    21 runs   (    0.58 ms per token,  1728.25 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   40941.36 ms /   455 runs   (   89.98 ms per token,    11.11 tokens per second)
llama_print_timings:       total time =   42003.18 ms /   456 tokens
Llama.generate: prefix-match hit


Response:  This text appears to be a portion of the Consumer Credit Reporting Agencies Act, specifically related to

Successfully processed complaint 333

Processing complaint 103/5

---Processing new complaint---
Input: conducted another investigation based new complaint submission able confirm time transaction questio...


...

Response: 

Category: Credit Card

Summary: The customer is expressing dissatisfaction with the credit card company's handling of a dispute regarding a transaction on their account. They believe they were wrongly accused of making an unauthorized purchase and are requesting to see proof of this allegation. However, they have been met with resistance from the company and feel disrespected by their communication. The customer also mentions that their credit report has been negatively affected due to the delinquency reporting related to this issue. They want the account closed as soon as possible and are frustrated with the overall experience.

Successfully processed complaint 103

Processing complaint 57/5

---Processing new complaint---
Input: american express reduced credit limit without valid justification advance notification spoke rep sta...



llama_print_timings:        load time =   10772.41 ms
llama_print_timings:      sample time =      84.78 ms /   136 runs   (    0.62 ms per token,  1604.11 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   29430.14 ms /   325 runs   (   90.55 ms per token,    11.04 tokens per second)
llama_print_timings:       total time =   30353.59 ms /   326 tokens
Llama.generate: prefix-match hit


Response: 

Category: Credit Card

Summary: The customer is complaining about American Express reducing their credit limit without prior notification and justification. They believe the decision was made due to an increase in balance on their account, but were later told it was because of their high utilization rate despite always maintaining a good standing account. The customer feels that American Express took advantage of them during a promotional period when they were paying significant amounts towards their balance and is seeking help from financial regulators as they are experiencing financial hardship due to unexpected circumstances such as illness or workplace closure. They also feel that the representative lacked proper training and handling skills, which added to their frustration.

Successfully processed complaint 57

Processing complaint 239/5

---Processing new complaint---
Input: block except otherwise provided section consumer reporting agency shall block reporting infor


llama_print_timings:        load time =   10772.41 ms
llama_print_timings:      sample time =      15.54 ms /    27 runs   (    0.58 ms per token,  1737.56 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   41149.33 ms /   455 runs   (   90.44 ms per token,    11.06 tokens per second)
llama_print_timings:       total time =   42244.79 ms /   456 tokens


Response:  This text appears to be a portion of a statute or regulation related to the blocking and unblocking of consumer credit reports due to

Successfully processed complaint 239


In [26]:
smaller_data['mistral_response']

,mistral_response
150,"Based on the given customer complaint, it app..."
332,This text appears to be a portion of the Cons...
102,\n\nCategory: Credit Card\n\nSummary: The cust...
56,\n\nCategory: Credit Card\n\nSummary: The cust...
238,This text appears to be a portion of a statut...


In [27]:
def extract_category(text):
    # Define the regex pattern to match "category:" or "Category:" followed by a word
    pattern = r'category:\s*(\w+)'  # The pattern itself remains the same

    # Use re.search with the re.IGNORECASE flag to make it case-insensitive
    match = re.search(pattern, text, re.IGNORECASE)

    # If a match is found, return the captured group, else return None
    if match:
        return match.group(1)
    else:
        pattern1 = r'(credit_card|retail_banking|credit_reporting|mortgages_and_loans|debt_collection)'
        match = re.search(pattern1, text, re.IGNORECASE)
        if match:
            return match.group()
        else:
            return ''

In [28]:
def mistral_response_cleaned(text):
    # Example of cleaning or extracting information from the text
    cleaned_text = text.lower()  # Example transformation
    return cleaned_text


In [29]:
smaller_data.head()

,product,narrative,summary,mistral_response,mistral_response_cleaned
150,credit_reporting,true identity theft victim identity theft info...,The input discusses an individual who has disc...,"Based on the given customer complaint, it app...","based on the given customer complaint, it app..."
332,credit_reporting,blockexcept otherwise provided section consume...,The text describes regulations on consumer rep...,This text appears to be a portion of the Cons...,this text appears to be a portion of the cons...
102,credit_card,conducted another investigation based new comp...,The text indicates an ongoing conflict related...,\n\nCategory: Credit Card\n\nSummary: The cust...,\n\ncategory: credit card\n\nsummary: the cust...
56,credit_card,american express reduced credit limit without ...,The user has complained about American Express...,\n\nCategory: Credit Card\n\nSummary: The cust...,\n\ncategory: credit card\n\nsummary: the cust...
238,credit_reporting,block except otherwise provided section consum...,The section explains the rules and procedures ...,This text appears to be a portion of a statut...,this text appears to be a portion of a statut...


In [30]:
print("\nExample comparison:")
print("\nOriginal response:")
print(smaller_data['mistral_response'].iloc[0])
print("\nCleaned response:")
print(smaller_data['mistral_response_cleaned'].iloc[0])


Example comparison:

Original response:
 Based on the given customer complaint, it appears that this is a case of Identity Theft. The customer mentions that they are a victim of identity theft and have found fraudulent transactions listed on their Equifax credit report. They also mention that they have been trying to get the issue resolved for some time but have encountered complications in the process.

Therefore, this complaint falls under the 'Others' category as it involves issues related to Identity Theft which is not directly linked to Credit Card, Bank Account Services, Loan or Mortgage services. However, since the identity theft has affected their credit report and ability to apply for new credit cards or loans in the future, it could potentially impact these areas as well.

The customer expresses frustration with the lengthy and complicated process of resolving the issue and the emotional and financial stress caused by the situation. They have also mentioned that they are see

##### **Q2.3: Calculate the F1 score** **(1 Marks)**

In [31]:
# Let's print each row's values to see what we're comparing
for idx, row in smaller_data.iterrows():
    print("\nRow", idx)
    print("Product (actual):", row['product'])
    print("Response (predicted):", row['mistral_response'][:100], "...")


Row 150
Product (actual): credit_reporting
Response (predicted):  Based on the given customer complaint, it appears that this is a case of Identity Theft. The custom ...

Row 332
Product (actual): credit_reporting
Response (predicted):  This text appears to be a portion of the Consumer Credit Reporting Agencies Act, specifically relat ...

Row 102
Product (actual): credit_card
Response (predicted): 

Category: Credit Card

Summary: The customer is expressing dissatisfaction with the credit card co ...

Row 56
Product (actual): credit_card
Response (predicted): 

Category: Credit Card

Summary: The customer is complaining about American Express reducing their  ...

Row 238
Product (actual): credit_reporting
Response (predicted):  This text appears to be a portion of a statute or regulation related to the blocking and unblocking ...


In [32]:
# Test category extraction on our responses
for idx, row in smaller_data.iterrows():
    extracted = extract_category(row['mistral_response'])
    print("\nRow", idx)
    print("Original product:", row['product'])
    print("Extracted category:", extracted)


Row 150
Original product: credit_reporting
Extracted category: 

Row 332
Original product: credit_reporting
Extracted category: 

Row 102
Original product: credit_card
Extracted category: Credit

Row 56
Original product: credit_card
Extracted category: Credit

Row 238
Original product: credit_reporting
Extracted category: 


In [33]:
def extract_category(text):
    # First try to match 'Category: X' pattern
    pattern = r'Category:\s*([\w\s]+)'
    match = re.search(pattern, text, re.IGNORECASE)

    if match:
        category = match.group(1).strip().lower()
        # Map extracted category to standard format
        if 'credit card' in category:
            return 'credit_card'
        if 'bank' in category:
            return 'retail_banking'
        if 'credit report' in category:
            return 'credit_reporting'
        if 'mortgage' in category:
            return 'mortgages_and_loans'
        if 'debt' in category:
            return 'debt_collection'

    # If no category label, try to 'infer' from content
    text = text.lower()
    if 'credit card' in text:
        return 'credit_card'
    if 'credit report' in text or 'credit reporting' in text:
        return 'credit_reporting'
    if 'mortgage' in text:
        return 'mortgages_and_loans'
    if 'debt' in text:
        return 'debt_collection'
    if 'bank' in text:
        return 'retail_banking'

    return ''  # Return empty string if no category found

In [34]:
smaller_data['extracted_categories'] = smaller_data['mistral_response'].apply(lambda x: extract_category(x))

In [35]:
f1 = f1_score(smaller_data['product'], smaller_data['extracted_categories'], average='micro')
print(f'F1 Score: {f1}')

F1 Score: 0.8


In [36]:
# Calculate F1 score for 'product' and 'mistral_response'
f1 =  f1_score(smaller_data['product'], smaller_data['mistral_response'],average='micro')

print(f'F1 Score: {f1}')

F1 Score: 0.0


In [37]:
# Calculate F1 score for 'product' and raw mistral_response
f1_raw = f1_score(smaller_data['product'],
                  smaller_data['mistral_response'].apply(lambda x: extract_category(x)),
                  average='micro')

# Calculate F1 score for 'product' and cleaned mistral_response
f1_cleaned = f1_score(smaller_data['product'],
                      smaller_data['mistral_response_cleaned'].apply(lambda x: extract_category(x)),
                      average='micro')

print(f'F1 Score (Raw Response): {f1_raw}')
print(f'F1 Score (Cleaned Response): {f1_cleaned}')

F1 Score (Raw Response): 0.8
F1 Score (Cleaned Response): 0.8


In [57]:
# Calculate F1 score for 'product' and 'mistral_response_cleaned'
f2 =  f1_score("product, "mistral_response_cleaned", average='micro')
print(f'F1 Score: {f2}')

SyntaxError: unterminated string literal (detected at line 2) (<ipython-input-57-b63fc6e4ba81>, line 2)

##### **Q2.4: Explain the difference in F1 scores between mistral_response and mistral_response_cleaned.** **(1 Marks)**

In the zero-shot text classification approach, I investigated the impact of text cleaning specifically, on category extraction by implementing two strategies.

* A basic lowercase conversion
* A more comprehensive cleaning method involving 'special' character removal and word filtering.

Both cleaning approaches yielded an identical F1 score of 0.8
The category extraction method demonstrated consistent performance across different cleaning techniques.

A few key insights include:

Simple lowercase conversion was just as effective as the more complex cleaning method.
Category extraction depends on 'meaningful' keyword associations rather than extensive text formatting.
Mistrals zero-shot classification model showed robust performance with minimal preprocessing  however, numerous challenges emerged managing the parameters compute performance which appeared to be lagging. Attempts to process the data in smaller batches and demonstrating a proof of concept with a single test while promising, leave a large space for improvement through trial and error, perhaps using quantization techniques to enhance the model.

The use of minimalized text prerocessing (lowercase conversion) enabled  the text to perform accurate, categorical extraction demonstrating the Mistral model can effectively, identify categories only requiring, minimal cleaning.


### **Question 3: Few-Shot Prompting for Text Classification (7 Marks)**

##### **Q3.1: Prepare examples for a few-shot prompt, formulate the prompt, and generate the Mistral response. (5 Marks)**

**Generate a set of gold examples by randomly selecting 10 instances of user_input and assistant_output from dataset ensuring a balanced representation with 2 examples from each class.**

In [39]:
import json
review_1 = data[data['product'] == 'credit_card']
review_2 = data[data['product'] == 'retail_banking']
review_3 = data[data['product'] == 'credit_reporting']
review_4 = data[data['product'] == 'mortgages_and_loans']
review_5 = data[data['product'] == 'debt_collection']

# Sample 2 examples for each category
examples_1 = review_1.sample(2, random_state=40)
examples_2 = review_2.sample(2, random_state=40)
examples_3 = review_3.sample(2, random_state=40)
examples_4 = review_4.sample(2, random_state=40)
examples_5 = review_5.sample(2, random_state=40)

# Concatenate examples for few shot prompting
examples_df = pd.concat([examples_1,examples_2,examples_3,examples_4,examples_5 ])

# Create the training set by excluding examples
gold_examples_df = data.drop(index=examples_df.index)

# Convert examples to JSON
columns_to_select = ['narrative', 'product']
examples_json = examples_df[columns_to_select].to_json(orient='records')

# Print the first record from the JSON
print(json.loads(examples_json)[0])

# Print the shapes of the datasets
print("Examples Set Shape:", examples_df.shape)
print("Gold Examples Shape:", gold_examples_df.shape)

{'narrative': 'called request new york state covid relief plan day interest fee waived amex provided relief leading late payment amex refused honor relief day gap insists charging late fee', 'product': 'credit_card'}
Examples Set Shape: (10, 3)
Gold Examples Shape: (490, 3)


- Define your **system_message**.
- Define **first_turn_template**, **example_template** and **prediction template**
- **create few shot prompt** using gold examples and system_message
- Randomly select 30 rows from test_df as test_data
- Create **mistral_response** with **mistral_response_cleaned** columns for this

In [40]:
system_message = system_message = "You are an expert at classifying customer complaints into specific financial product categories. Use the following examples to understand the classification patterns and accurately categorize new customer complaints into one of these categories: 'Credit Card', 'Bank Account Services', 'Loan', 'Mortgage', and 'Others'."

In [58]:
# Template for few-shot learning
first_turn_template = """<s>[INST] <<SYS>>
{system_message}
<</SYS>>```{user_message}``` [/INST]
{assistant_message}
</s>"""

examples_template = """<s>[INST] ```{user_message}``` [/INST]
{assistant_message}
</s>"""

prediction_template = """<s>[INST] ```{user_message}``` [/INST]"""

def generate_prompt(few_shot_prompt, new_review):
    prompt = few_shot_prompt + prediction_template.format(user_message=new_review)
    return prompt

In [59]:
def create_few_shot_prompt(system_message, examples_df):

    """
    Return a prompt message in the format expected by Mistral 7b.
    10 examples are selected randomly as golden examples to form the
    few-shot prompt.
    We then loop through each example and parse the narrative as the user message
    and the product as the assistant message.

    Args:
        system_message (str): system message with instructions for classification
        examples(DataFrame): A DataFrame with examples (product + narrative + summary)
        to form the few-shot prompt.

    Output:
        few_shot_prompt (str): A prompt string in the Mistral format
    """

    few_shot_prompt = ''

    columns_to_select = ['narrative', 'product']
    examples = (
        examples_df.loc[:, columns_to_select].to_json(orient='records')
    )

    for idx, example in enumerate(json.loads(examples)):
        user_input_example = example['narrative']
        assistant_output_example = example['product']

        if idx == 0:
            few_shot_prompt += first_turn_template.format(
                system_message=system_message,
                user_input=user_input_example,
                assistant_output=assistant_output_example
            )
        else:
            few_shot_prompt += examples_template.format(
                user_input=user_input_example,
                assistant_output=assistant_output_example
            )

    return few_shot_prompt

In [60]:
few_shot_prompt = """<s>[INST] <<SYS>>
You are an expert at classifying customer complaints into specific financial product categories. Use the following examples to understand the classification patterns and accurately categorize new customer complaints into one of these categories: 'Credit Card', 'Bank Account Services', 'Loan', 'Mortgage', and 'Others'.
<</SYS>>

Examples:
1. User: "My credit card was charged twice for a single transaction and the bank is not resolving it."
Assistant: credit_card

2. User: "My checking account was frozen without any notice and I can't access my funds."
Assistant: retail_banking

3. User: "There are errors in my credit report that I've disputed multiple times."
Assistant: credit_reporting

4. User: "The mortgage company increased my monthly payments without proper explanation."
Assistant: mortgages_and_loans

5. User: "Collection agency keeps calling about a debt I've already paid off."
Assistant: debt_collection[/INST]"""

In [61]:
print(few_shot_prompt)

<s>[INST] <<SYS>>
You are an expert at classifying customer complaints into specific financial product categories. Use the following examples to understand the classification patterns and accurately categorize new customer complaints into one of these categories: 'Credit Card', 'Bank Account Services', 'Loan', 'Mortgage', and 'Others'.
<</SYS>>

Examples:
1. User: "My credit card was charged twice for a single transaction and the bank is not resolving it."
Assistant: credit_card

2. User: "My checking account was frozen without any notice and I can't access my funds." 
Assistant: retail_banking

3. User: "There are errors in my credit report that I've disputed multiple times."
Assistant: credit_reporting

4. User: "The mortgage company increased my monthly payments without proper explanation."
Assistant: mortgages_and_loans

5. User: "Collection agency keeps calling about a debt I've already paid off."
Assistant: debt_collection[/INST]


In [67]:
def generate_prompt(few_shot_prompt, new_review):
    prompt = few_shot_prompt + prediction_template.format(user_message=new_review)
    return prompt

In [68]:
def generate_mistral_response(input_text):
    prompt = generate_prompt(few_shot_prompt, input_text)
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1200,
        temperature=0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=["/s"],
        echo=False
    )

    return response["choices"][0]["text"]

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    print(response_text)
    return response_text

In [69]:
# Randomly select 50 rows from gold_examples

#1 Create the training set by excluding examples used for few-shot prompting
gold_examples_df = data.drop(index=examples_df.index)

#2 Randomly select 50 rows from remaining data
new_data = gold_examples_df.sample(n=50, random_state=40)

In [72]:
# Reinitalize the model with larger context window
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=4,
    n_batch=32,
    n_gpu_layers=20,
    n_ctx=2048  # Increase context window
)

# Test with 'one' complaint to make sure it works
test_narrative = new_data['narrative'].iloc[0]
print("Testing with narrative:", test_narrative[:100], "...")

test_response = generate_mistral_response(test_narrative)
print("\nResponse:", test_response)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loa

Testing with narrative: anyone us credit report another consumer report deny credit must tell give name address phone number ...



llama_print_timings:        load time =    1323.53 ms
llama_print_timings:      sample time =     596.64 ms /   991 runs   (    0.60 ms per token,  1660.97 tokens per second)
llama_print_timings: prompt eval time =   16712.56 ms /   901 tokens (   18.55 ms per token,    53.91 tokens per second)
llama_print_timings:        eval time =   93794.60 ms /   990 runs   (   94.74 ms per token,    10.55 tokens per second)
llama_print_timings:       total time =  114912.89 ms /  1891 tokens



Response:  Based on the provided text, it appears that there are multiple complaints mentioned in this conversation. Here's how I would categorize each complaint based on the context:

1. Credit Card: "My credit card was charged twice for a single transaction and the bank is not resolving it." - This complaint falls under the category of credit cards as it specifically mentions a credit card being charged twice without authorization, which is a common issue related to credit card transactions.
2. Bank Account Services (Retail Banking): "My checking account was frozen without any notice and I can't access my funds." - This complaint falls under the category of retail banking as it mentions a checking account being frozen without prior notification, which is an issue that typically relates to bank account services or customer service.
3. Credit Report: "There are errors in my credit report that I've disputed multiple times" and "feel still fraudulent activity score cleared identity thef

In [73]:
# Test with short input
test_text = "This is a credit card dispute."
response = lcpp_llm(
    prompt=test_text,
    max_tokens=50,  # Much smaller output
    temperature=0,
    echo=False
)
print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1323.53 ms
llama_print_timings:      sample time =      26.82 ms /    50 runs   (    0.54 ms per token,  1863.93 tokens per second)
llama_print_timings: prompt eval time =     263.10 ms /     7 tokens (   37.59 ms per token,    26.61 tokens per second)
llama_print_timings:        eval time =    4398.11 ms /    49 runs   (   89.76 ms per token,    11.14 tokens per second)
llama_print_timings:       total time =    4836.36 ms /    56 tokens


{'id': 'cmpl-1a18912b-81ce-48ff-8b5f-b5f7dc31999d', 'object': 'text_completion', 'created': 1737597195, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q5_K_M.gguf', 'choices': [{'text': ' I made a purchase on Amazon using my Capital One Venture Rewards Card on 12/30/2018 for $54.99. The item was delivered on 1/1/2019', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 8, 'completion_tokens': 50, 'total_tokens': 58}}


In [ ]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
new_data['mistral_response'] = new_data['narrative'].apply(lambda x: generate_mistral_response(x))

In [77]:
# Create the response column
new_data['mistral_response'] = None

# Process in small batches with progress tracking / indexing
batch_size = 5
total_rows = len(new_data)
total_batches = (total_rows + batch_size - 1) // batch_size
responses = []

for start_idx in range(0, total_rows, batch_size):
    print(f"\nProcessing batch {start_idx//batch_size + 1} of {total_batches}")

    # Get the end index for this batch
    end_idx = min(start_idx + batch_size, total_rows)
    batch = new_data.iloc[start_idx:end_idx]

    batch_responses = []
    for _, row in batch.iterrows():
        print(f"Processing complaint...")
        try:
            truncated_text = row['narrative'][:500]
            response = lcpp_llm(
                prompt=truncated_text,
                max_tokens=200,
                temperature=0,
                echo=False
            )
            batch_responses.append(response["choices"][0]["text"])
        except Exception as e:
            print(f"Error processing complaint: {e}")
            batch_responses.append(None)

    # Update the dataframe directly for this batch
    for i, response in enumerate(batch_responses):
        new_data.iloc[start_idx + i, new_data.columns.get_loc('mistral_response')] = response

    print(f"Completed batch {start_idx//batch_size + 1}")

print("\nProcessing complete!")


Processing batch 1 of 10
Processing complaint...


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1323.53 ms
llama_print_timings:      sample time =     119.47 ms /   200 runs   (    0.60 ms per token,  1674.00 tokens per second)
llama_print_timings: prompt eval time =    1837.59 ms /    89 tokens (   20.65 ms per token,    48.43 tokens per second)
llama_print_timings:        eval time =   17993.96 ms /   199 runs   (   90.42 ms per token,    11.06 tokens per second)
llama_print_timings:       total time =   20604.58 ms /   288 tokens
Llama.generate: prefix-match hit


Processing complaint...



llama_print_timings:        load time =    1323.53 ms
llama_print_timings:      sample time =     112.00 ms /   200 runs   (    0.56 ms per token,  1785.63 tokens per second)
llama_print_timings: prompt eval time =    1597.29 ms /    63 tokens (   25.35 ms per token,    39.44 tokens per second)
llama_print_timings:        eval time =   17937.70 ms /   199 runs   (   90.14 ms per token,    11.09 tokens per second)
llama_print_timings:       total time =   20295.44 ms /   262 tokens
Llama.generate: prefix-match hit


Processing complaint...


...

Processing complaint...



llama_print_timings:        load time =    1323.53 ms
llama_print_timings:      sample time =      84.77 ms /   200 runs   (    0.42 ms per token,  2359.38 tokens per second)
llama_print_timings: prompt eval time =    1567.04 ms /    63 tokens (   24.87 ms per token,    40.20 tokens per second)
llama_print_timings:        eval time =   17940.17 ms /   199 runs   (   90.15 ms per token,    11.09 tokens per second)
llama_print_timings:       total time =   20203.38 ms /   262 tokens
Llama.generate: prefix-match hit


Processing complaint...



llama_print_timings:        load time =    1323.53 ms
llama_print_timings:      sample time =     111.79 ms /   200 runs   (    0.56 ms per token,  1789.00 tokens per second)
llama_print_timings: prompt eval time =    1598.91 ms /    63 tokens (   25.38 ms per token,    39.40 tokens per second)
llama_print_timings:        eval time =   17867.70 ms /   199 runs   (   89.79 ms per token,    11.14 tokens per second)
llama_print_timings:       total time =   20200.14 ms /   262 tokens


Completed batch 10

Processing complete!


In [78]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
new_data['mistral_response_cleaned'] = new_data['mistral_response'].apply(lambda x: mistral_response_cleaned(x))

##### **Q3.2: Calculate the F1 score** **(1 Marks)**

In [79]:
# Calculate F1 score for 'product' and 'mistral_response'
f1 = f1_score(new_data['product'],
              new_data['mistral_response'].apply(lambda x: extract_category(x)),
              average='micro')

print(f'F1 Score: {f1}')

F1 Score: 0.54


##### **Q3.3: Share your observations on the few-shot and zero-shot prompt techniques. (1 Marks)**

This few-shot example using the Mistral model achieved an F1 score of 0.54, which is lower than the previous zero-shot approach that scored 0.8.

The decline in performance can be attributed to (potential?) limitations of the few-shot architecture.

With zero-shot, the model used pre-trained knowledge and generalized understanding of financial complaint categories and introduced a few, specific examples. This may be introducing bias / noise leading towards  sparse interpretation or alternatively mean, better performance.
Potential next steps could include refining the few-shot example selection, experimenting with different prompt structures, or exploring alternative few-shot learning techniques such as cross checking the performance against another LLM such as GPT.

# **Section 3: Text to Text generation**

### **Question 4: Zero-Shot Prompting for Text Summarization (5 Marks)**

##### **Q4.1: Define the Prompt Template, System Message, generate prompt and model response** **(3 Marks)**


- Define a **system message** as a string and assign it to the variable system_message to generate summary of narrative in data.
- Create a **zero shot prompt template** that incorporates the system message and user input.
- Define **generate_prompt** function that takes both the system_message and user_input as arguments and formats them into a prompt template


Write a Python function called **generate_mistral_response** that takes a single parameter, narrative, which represents the user's complain. Inside the function, you should perform the following tasks:


- **Combine the system_message and narrative to create a prompt string using generate_prompt function.**

*Generate a response from the Mistral model using the lcpp_llm instance with the following parameters:*

- prompt should be the combined prompt string.
- max_tokens should be set to 1200.
- temperature should be set to 0.
- top_p should be set to 0.95.
- repeat_penalty should be set to 1.2.
- top_k should be set to 50.
- stop should be set as a list containing '/s'.
- echo should be set to False.
Extract and return the response text from the generated response.

Don't forget to provide a value for the system_message variable before using it in the function.

In [80]:
system_message = "Provide a concise, clear summary of the financial customer complaint, capturing the key issues, parties involved, and main concerns without unnecessary details."

In [81]:
zero_shot_prompt_template = """<s>[INST] <<SYS>>
{system_message}
<</SYS>>

Summarize the following customer complaint:
```{user_message}```
[/INST]"""

def generate_prompt(system_message, user_input):
    prompt = zero_shot_prompt_template.format(system_message=system_message, user_message=user_input)
    return prompt

def generate_mistral_response(input_text):

    # Combine user_prompt and system_message to create the prompt
    prompt = generate_prompt(system_message, input_text)

    # Define the Llama model along with its parameters for generating a response
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1200,
        temperature=0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=["/s"],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    print(response_text)
    return response_text

##### **Q4.2: Generate mistral_response column containing LLM generated summaries** **(1 Marks)**

In [82]:
# Randomly select 30 rows
gold_examples = data.sample(n=30 , random_state=40)

In [83]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
gold_examples['mistral_response'] = gold_examples['narrative'].apply(lambda x: generate_mistral_response(x))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1323.53 ms
llama_print_timings:      sample time =      96.89 ms /   157 runs   (    0.62 ms per token,  1620.43 tokens per second)
llama_print_timings: prompt eval time =    4533.21 ms /   273 tokens (   16.61 ms per token,    60.22 tokens per second)
llama_print_timings:        eval time =   14243.44 ms /   156 runs   (   91.30 ms per token,    10.95 tokens per second)
llama_print_timings:       total time =   19418.96 ms /   429 tokens
Llama.generate: prefix-match hit


 A customer discovered a fraudulent charge of undisclosed amount on their Capital One checking account, which was quickly canceled. The customer disputed the charge and received provisional credit pending determination. However, the claim was denied by Capital One despite several appeals and communications between the parties. The customer believes that someone intercepted and activated their debit card without authorization to make a fraudulent purchase. They also reported this incident to local police. Despite not receiving the replacement card yet, Capital One sent it and allowed its use for credit purchases without prior consent. The customer is concerned about the bank's handling of the situation and believes there has been malfeasance and gross business practices involved in sending a replacement card without authorization and allowing unauthorized usage.



llama_print_timings:        load time =    1323.53 ms
llama_print_timings:      sample time =     184.63 ms /   298 runs   (    0.62 ms per token,  1614.00 tokens per second)
llama_print_timings: prompt eval time =    9131.13 ms /   507 tokens (   18.01 ms per token,    55.52 tokens per second)
llama_print_timings:        eval time =   27537.21 ms /   297 runs   (   92.72 ms per token,    10.79 tokens per second)
llama_print_timings:       total time =   37937.99 ms /   804 tokens
Llama.generate: prefix-match hit


 A customer filed a complaint regarding identity theft, requesting a credit reporting agency to block certain information from their report. The dispute arose when the agency failed to promptly block the disputed information following receipt of an identity theft report. The consumer is concerned about potential damage to their credit score due to erroneous information being reported.

The Consumer Reporting Agency (CRA) has the authority to decline or rescind such blocks, but must provide clear reasons for doing so and notify both the consumer and any relevant furnishers of the dispute. The CRA may also be required to block information relating to a transaction that resulted from identity theft.

The CRA is obligated to promptly notify affected consumers when information has been reinstated, and must provide them with clear notice containing specific details about the disputed information. Resellers are also subject to these requirements if they file consumer reports concerning the id


llama_print_timings:        load time =    1323.53 ms
llama_print_timings:      sample time =      84.94 ms /   142 runs   (    0.60 ms per token,  1671.87 tokens per second)
llama_print_timings: prompt eval time =    4024.19 ms /   184 tokens (   21.87 ms per token,    45.72 tokens per second)
llama_print_timings:        eval time =   12842.61 ms /   141 runs   (   91.08 ms per token,    10.98 tokens per second)
llama_print_timings:       total time =   17435.28 ms /   325 tokens
Llama.generate: prefix-match hit


...

 The customer is a victim of identity theft, with unauthorized transactions appearing on their Equifax credit report. They have been trying to resolve the issue for an extended period and find it complicated and time-consuming. The customer believes that these issues stem from the 2017 Equifax data breach. They are concerned about negative information affecting their ability to apply for credit, including being denied a first-time credit card application. Additionally, they mention that the process has caused significant financial and emotional stress, and in some cases, have prevented them from getting a job due to adverse actions based on incorrect reporting. The customer is currently working with an attorney specializing in predatory loans and breach victim compensation while carefully reviewing their credit file for any discrepancies related to the disputed account.



llama_print_timings:        load time =    1323.53 ms
llama_print_timings:      sample time =     183.00 ms /   298 runs   (    0.61 ms per token,  1628.41 tokens per second)
llama_print_timings: prompt eval time =    8369.72 ms /   507 tokens (   16.51 ms per token,    60.58 tokens per second)
llama_print_timings:        eval time =   27472.11 ms /   297 runs   (   92.50 ms per token,    10.81 tokens per second)
llama_print_timings:       total time =   37022.92 ms /   804 tokens
Llama.generate: prefix-match hit


 A customer filed a complaint regarding identity theft, requesting a credit reporting agency to block certain information from their report. The dispute arose when the agency failed to promptly block the disputed information following receipt of an identity theft report. The consumer is concerned about potential damage to their credit score due to erroneous information being reported.

The Consumer Reporting Agency (CRA) has the authority to decline or rescind such blocks, but must provide clear reasons for doing so and notify both the consumer and any relevant furnishers of the dispute. The CRA may also be required to block information relating to a transaction that resulted from identity theft.

The CRA is obligated to promptly notify affected consumers when information has been reinstated, and must provide them with clear notice containing specific details about the disputed information. Resellers are also subject to these requirements if they file consumer reports concerning the id


llama_print_timings:        load time =    1323.53 ms
llama_print_timings:      sample time =      70.27 ms /   116 runs   (    0.61 ms per token,  1650.71 tokens per second)
llama_print_timings: prompt eval time =    1877.08 ms /    96 tokens (   19.55 ms per token,    51.14 tokens per second)
llama_print_timings:        eval time =   10318.47 ms /   116 runs   (   88.95 ms per token,    11.24 tokens per second)
llama_print_timings:       total time =   12618.37 ms /   212 tokens


 A mother called to report an unauthorized user was removed from her credit line, which negatively affected her credit score. The party involved is Experian, the credit reporting agency. Her main concern is that she needs a good credit score to refinance her home as her husband recently lost his job and she's having difficulty making ends meet with just her nurse salary. She tried disputing the issue online through Experian but was unable to get in touch with a representative. She requests assistance in getting Experian to reflect the proper credit score change as soon as possible.


##### **Q4.3: Evaluate bert score** **(1 Marks)**

In [84]:
def evaluate_score(test_data, scorer, bert_score=False):

    """
    Return the ROUGE score or BERTScore for predictions on gold examples
    For each example we make a prediction using the prompt.
    Gold summaries and the AI generated summaries are aggregated into lists.
    These lists are used by the corresponding scorers to compute metrics.
    Since BERTScore is computed for each candidate-reference pair, we take the
    average F1 score across the gold examples.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        scorer (function): Scorer function used to compute the ROUGE score or the
                           BERTScore
        bert_score (boolean): A flag variable that indicates if BERTScore should
                              be used as the metric.

    Output:
        score (float): BERTScore or ROUGE score computed by comparing model predictions
                       with ground truth
    """

    model_predictions = test_data['mistral_response'].tolist()
    ground_truths = test_data['summary'].tolist()
    if bert_score:
        score = scorer.compute(
            predictions=model_predictions,
            references=ground_truths,
            lang="en",
            rescale_with_baseline=True
        )

        return sum(score['f1'])/len(score['f1'])
    else:
        return scorer.compute(
            predictions=model_predictions,
            references=ground_truths
        )

In [85]:
# Load the BERTScore evaluator
bert_scorer = evaluate.load("bertscore")

In [86]:
# Calculate BERTScore
score = evaluate_score(
    gold_examples,
    bert_scorer,
    bert_score=True
)

print(f'BERTScore: {score}')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: 0.3167718638976415


The BERTScore - 0.317 is relatively low since BERTScore has ranges from 0 to 1, with higher values indicating greater similarity between the generated and reference summaries. A score of 0.317 suggests the model - generated summaries could be struggling with finding the 'semantic similarity' to the original summaries.

There could also be differences or variations in how the model is captururing and interpreting key sections of the narratives compared to the human originated summaries.